### Выгрузка данных

In [1]:
import pandas as pd
import numpy as np
import warnings
import math
import json
import ast

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
class FilledDataset:
    def __init__(self, data):

        dt = ast.literal_eval(str(data))

        self.df = pd.DataFrame(dt['places']).transpose()
        self.df.loc[self.df['preview_text'] ==
                    '', 'preview_text'] = self.df['title']

        self.df = self.df.dropna(subset=['id', 'coordinates'])
        self.distance_threshold = 0.00215

        self.stops_coordinates = self.df.loc[self.df['type'] ==
                                             'Остановка']['coordinates'].apply(self.replace_coord)
        self.hire_coordinates = self.df.loc[self.df['type'] == 'Прокат']['coordinates'].apply(
            self.replace_coord)

        self.df.tickets = self.df.tickets.apply(
            lambda y: np.nan if y == np.nan or y[0] == '' else y)

    def time_handler(self, val):

        days = {'Пн': np.nan, 'Вт': np.nan, 'Ср': np.nan,
                'Чт': np.nan, 'Пт': np.nan, 'Сб': np.nan, 'Вс': np.nan}
        decoder = {'0': 'Пн', '1': 'Вт', '2': 'Ср',
                   '3': 'Чт', '4': 'Пт', '5': 'Сб', '6': 'Вс'}

        if len(val) == 0:
            return np.nan

        for i, p in enumerate(val):

            if p['left'] == 'Ежедневно':
                for key in days.keys():
                    days[key] = p['right']

            elif p['left'] == 'Будни':
                for j, key in enumerate(days.keys()):
                    if j in [0, 1, 2, 3, 4]:
                        days[key] = p['right']

            elif len(p['left']) == 7 and p['left'][3] == '—' and p['right'] not in ['Выходной', 'По записи']:
                exact_days = [index for index in range(list(days.keys()).index(p['left'].split()[0]),
                              list(days.keys()).index(p['left'].split()[2]) + 1)]

                for k in exact_days:
                    days[decoder[str(k)]] = p['right']

            elif len(p['left']) == 6 and len(p['left'].split(',')) == 2 and p['right'] != 'Выходной':
                days[p['left'][-2:]] = p['right']
                days[p['left'][:2]] = p['right']

            elif len(p['left']) == 2 and p['right'] != 'Выходной':
                days[p['left']] = p['right']

        if list(days.values()) == [np.nan for i in range(7)]:
            return np.nan

        return days

    def free_flag(self, x):
        if x is np.nan:
            return 1
        else:
            return 0

    def closed_flag(self, x):
        try:
            if x[0]['left'] == 'Временно закрыт' or x[0]['left'] == 'Павильон находится на реставрации.':
                return np.nan
            else:
                return 1
        except:
            return 1

    def check_stops(self, dot):
        for coordinates in self.stops_coordinates:
            if math.hypot(dot[1] - coordinates[1], dot[0] - coordinates[0]) <= self.distance_threshold:
                return 1
        return 0

    def check_hire(self, dot):
        for coordinates in self.hire_coordinates:
            if math.hypot(dot[1] - coordinates[1], dot[0] - coordinates[0]) <= self.distance_threshold:
                return 1
        return 0

    def replace_coord(self, x):
        return [x[1], x[0]]
    
    def replace_24(self, x):
    
        if x is np.nan:
            return np.nan

        for key in x.keys():
            if x[key] is np.nan:
                continue
            if x[key][-5:] == '24:00':

                s = x[key][:8] + '23:59'
                x.update({key: s})
        return x

    def set_new_columns(self):

        self.df['coordinates'] = self.df['coordinates'].apply(
            self.replace_coord)

        self.df['work_schedule'] = self.df['schedule'].apply(self.time_handler)
        self.df['work_schedule'] = self.df['work_schedule'].apply(self.replace_24)
        self.df['electrobus_flag'] = self.df['coordinates'].apply(
            self.check_stops)
        self.df['scooter_flag'] = self.df['coordinates'].apply(self.check_hire)
        self.df['closed'] = self.df['schedule'].apply(self.closed_flag)
        self.df['free'] = self.df['tickets'].apply(self.free_flag)
        self.df.dropna(subset='closed', inplace=True)
        self.df.drop(columns=['closed'], inplace=True)

        self.df.drop(self.df[self.df['title'].isin([
            'Спортивный магазин Fitness Place', 'Самый большой каток Москвы', 'Банкетный зал «Выставка»',
            'Каток в парке «Останкино»', 'Концертный зал Главного ботанического сада имени Н.В. Цицина РАН',
            'Тюбинговая горка', 'Лодочная станция «Алые\xa0паруса»', 'Павильон №47. Дом ремесел ',
            'Комплекс спортивных площадок в аренду', 'Площадка для бадминтона и настольного тенниса',
            'Московская канатная дорога на Воробьевых горах', 'Павильон №48. «Главпиво»', 'Павильон №9. Театр сказок',
            'Теннисный клуб «Первый\xa0корт»', 'Вейк-парк DROP PARK VDNH', 'Павильон №67. Карелия',
            'Смотровая площадка PANORAMA360', 'Северная петля Кольцевой дороги', 'Павильон №21',
            'Аттракцион Zipline в экстрим-парке Skypark Moscow', 'Прокат беговых лыж', 'Прокат беговых лыж', 
            'Прокат беговых лыж', 'Прокат детских квадроциклов и фэтбайков', 'Вендинговый аппарат', 
            'Велопрокат', 'Велопрокат', 'Велопрокат', 'Велопрокат', 'Истребитель Су-27'])].index, inplace=True)
        self.df.drop(self.df[(self.df['work_schedule'].isna())
                             & (self.df['cat'] == 'event')].index, inplace=True)

    def get_dataset(self):
        self.set_new_columns()
        return self.df

In [3]:
# Opening JSON file
with open('export (1).json') as json_file:
    data = json.load(json_file)
    
class_ex = FilledDataset(data)
df = class_ex.get_dataset()

### Разметка объектов

In [4]:
df.loc[df['type'].isin(['Туалеты', 'Прокат', 'Остановка', 'Билеты', 'Въезд', 'Вендинговый аппарат',
                       'Такси', 'Сувениры', 'Парковка', 'Банкомат', 'Вход', 'Инфоцентр', 'Храм',
                        'Образование', 'Пикник', 'Лекции и мастер-классы', 'Квест', 'Еда']), 'rating'] = 3000

df.loc[df['type'].isin(['Экскурсия', 'Читальня']), 'rating'] = 2000

df.loc[df['type'].isin(['Фонтан', 'Памятник', 'Пруд',
                       'Концерты и шоу', 'Комбо-программа']), 'rating'] = 1500

df.loc[df['type'].isin(
    ['Детская площадка', 'Зеленая зона', 'Спорт']), 'rating'] = 1000

df.loc[df['type'].isin(['Развлечения', 'Павильон', 'Выставка',
                       'Мастерская', 'Музей', 'Аттракцион']), 'rating'] = 500

df.loc[df['title'].isin(['Автобусные туры City Sightseeing', 'Строение 457', 'Павильон №16',
                        'Дворец бракосочетания', 'Павильон №49. Резиденция «Каскад + digital» при поддержке Музея Москвы',
                         'Стадион в парке «Останкино»', 'Павильон №19. «Атом»', 'Лодочная станция на Останкинском пруду',
                         'Павильон №33', 'Строение 313', 'Строение 84а', 'Театральная студия «Чайка»', 
                         'Фотостудия BestMemories', 'Мастерская высокой печати sixh', 'Летний кинотеатр-лекторий',
                         'Арт-лофт «Чудеса»', 'Зеленый театр', 'Школа эстрадного вокала «Голоса»',  'Павильон №46',
                        'Павильон №27. Физкультура и спорт',
                        'Памятный знак «Работникам ВСХВ, погибшим на фронтах Великой Отечественной войны 1941-1945 гг.»',
                        'Читальня', 'Памятник В.И. Ленину',
       'Памятник изобретателю телевизора В.К. Зворыкину',
       'Памятник И.В. Мичурину',]),
       'rating'] = 5000

df.loc[df['title'].isin(['Зеленый бельведер в парке «Останкино»', 'Арка Южного входа',
                         'Арка Северного входа', 'Сирингарий',
                        'Южный розарий', 'Северный розарий']), 'rating'] = 1000

df.loc[df['title'].isin(['Арка Главного входа', 'Фонтан «Дружба народов»', 'Павильон №34. Центр «Космонавтика и авиация»',
                        'Музей ВДНХ', 'Музей кино']), 'rating'] = 100

In [5]:
not_for_children = [
    'Павильон №58. Музей славянской письменности «Слово»', 'Павильон «Рабочий и колхозница»',
    'Павильон №5. Музей городского хозяйства Москвы', 'Дом культуры', 'Павильон «Книги» РОСИЗО',
    'Шахматный клуб', 'Павильон №13. Музей Востока', 'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
    'Павильон №30. Центр современных биотехнологий Музей «Биотех»', 'Павильон №57. «Россия — моя история»',
    'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
    'Павильон №18. Выставочно-торговый центр Республики Беларусь', 'Павильон №67. Карелия',
    'Павильон №68. Выставочно-торговый центр Республики Армения',
    'Павильон №10. Выставочно-торговый центр Республики Молдова',
    'Павильон №11. Выставочно-торговый центр Республики Казахстан ', 'Павильон №16', 'Павильон №25 «Нефть»',
    'Центр креативного арт-дизайна La Atmosfera',
    'Строение 457', 'Павильон №71. Дворец госуслуг (МФЦ) ', 'Sky Town. Высотный веревочный парк',
    'Павильон №48. «Главпиво»', 'Павильон №60. Потребительская кооперация',
    'Павильон №49. Резиденция «Каскад + digital» при поддержке Музея Москвы', 'Стадион в парке «Останкино»',
    'Читальня', 'Площадка для воркаута рядом со стадионом', 'Площадка для воркаута',
    'Площадка для воркаута рядом со стадионом', 'Концертный зал Главного ботанического сада имени Н.В. Цицина РАН',
    'Медиа Школа Ольги Спиркиной Ostankino.tv', 'Мемориал-стела «Врата победы»',
    'Памятник изобретателю телевизора В.К. Зворыкину', 'Памятник В.И. Ленину', 'Памятник И.В. Мичурину',
    'Памятный знак «Работникам ВСХВ, погибшим на фронтах Великой Отечественной войны 1941-1945 гг.»',
    'Павильон №12', 'Павильон №21', 'Зал для чтения']

not_for_date = [
    'Павильон №58. Музей славянской письменности «Слово»', 'Павильон «Рабочий и колхозница»',
    'Павильон №5. Музей городского хозяйства Москвы', 'Дом культуры', 'Павильон «Книги» РОСИЗО',
    'Шахматный клуб', 'Павильон №13. Музей Востока', 'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
    'Павильон №18. Выставочно-торговый центр Республики Беларусь', 'Павильон №67. Карелия',
    'Павильон №68. Выставочно-торговый центр Республики Армения',
    'Павильон №10. Выставочно-торговый центр Республики Молдова',
    'Павильон №11. Выставочно-торговый центр Республики Казахстан ', 'Павильон №16', 'Павильон №25 «Нефть»',
    'Строение 457', 'Павильон №71. Дворец госуслуг (МФЦ) ',
    'Павильон №60. Потребительская кооперация',
    'Павильон №49. Резиденция «Каскад + digital» при поддержке Музея Москвы', 'Стадион в парке «Останкино»',
    'Читальня', 'Площадка для воркаута рядом со стадионом', 'Площадка для воркаута',
    'Площадка для воркаута рядом со стадионом', 'Концертный зал Главного ботанического сада имени Н.В. Цицина РАН',
    'Медиа Школа Ольги Спиркиной Ostankino.tv', 'Мемориал-стела «Врата победы»',
    'Памятник изобретателю телевизора В.К. Зворыкину', 'Памятник В.И. Ленину', 'Памятник И.В. Мичурину',
    'Памятный знак «Работникам ВСХВ, погибшим на фронтах Великой Отечественной войны 1941-1945 гг.»',
    'Павильон №12', 'Павильон №21', 'Зал для чтения', 'Детская площадка с беседками на Липовой аллее',
    'Детская площадка в парке «Останкино»', 'Детская площадка между Липовой и Спортивной аллеями',
    'Детская площадка для занятий паркуром', 'Нефтеград. Детская интерактивная площадка',
    'Универсальная спортивная площадка для игры в мини-футбол и стритбол', 'Павильон №57. «Россия — моя история»',
    'Игровая площадка, адаптированная для детей с ограниченными возможностями',
    'Фантастический город. Детская площадка', 'Детская площадка за павильоном №34', 'Космос. Детская площадка',
    '«Детское посольство»', 'Игровая площадка Capital Group', 'Студия детских настольных игр Dodoland']

not_for_family = not_for_children

for_animals = ['Павильон №34. Центр «Космонавтика и авиация»',
       'Павильон №58. Музей славянской письменности «Слово»',
       'Павильон «Рабочий и колхозница»',
       'Павильон №5. Музей городского хозяйства Москвы',
       'Павильон №36. Музей кино',
       'Павильон №53-54. Музей Гаража особого назначения ФСО России',
       'Интерактивный музейный комплекс «Буран»','Павильон №17. Галерея «Лес»',
       'Арка Главного входа', 'Павильон №2. Робостанция',
       'Павильон №26. Музей Транспорта Москвы',
       'Павильон №13. Музей Востока ',
       'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
       'Павильон №30. Центр современных биотехнологий Музей «Биотех»',
       'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
       'Павильон №18. Выставочно-торговый центр Республики Беларусь',
       'Павильон №62. Международный центр балета',
       'Павильон №68. Выставочно-торговый центр Республики Армения',
       'Экспозиция экосистем ', 'Экотропа',
       '«Техноград». Инновационно-образовательный комплекс',
       'Павильон «Умный город»',
       'Павильон №10. Выставочно-торговый центр Республики Молдова',
       'Павильон №11. Выставочно-торговый центр Республики Казахстан ',
       'Павильон №25 «Нефть»',
       'Павильон №29. Эколого-просветительский центр «Цветоводство»',
       'Павильон №23. «Москвариум»',
       'Павильон №71. Дворец госуслуг (МФЦ) ',
       'Павильон №40. Школа пекарей',
       'Павильон №60. Потребительская кооперация',
       'Фонтан «Мальчик с рыбой»', 'Фонтан «Каменный цветок»',
       'Фонтан «Золотой колос»', 'Фонтан «Дружба народов»',
       'Нефтеград. Детская интерактивная площадка', 'Зеленый лабиринт',
       'Понтонный променад',
       'Зеленый бельведер в парке «Останкино»',
       'Большая прогулочная аллея', 'Большая картина полей',
       'Площадка для воркаута',
       'Площадка для воркаута рядом со стадионом',
       'Дворцовый (Останкинский) пруд',
       'Каскад Каменских прудов (2-й пруд)',
       'Каскад Каменских прудов (3-й пруд)',
       'Каскад Каменских прудов (4-й пруд)', 'Садовый пруд',
       'Северный розарий', 'Сквер юннатов',
       'Универсальная спортивная площадка для игры в мини-футбол и стритбол',
       'Дубовая роща',
       'Мичуринский сад', 'Медовый домик',
       'Латвийский сквер', 'Летняя эстрада в парке «Останкино»',
       'Детская площадка на главной аллее парка «Останкино»',
       'Мемориал-стела «Врата победы»', 'Макет ракеты-носителя «Восток»',
       'Игровая площадка, адаптированная для детей с ограниченными возможностями', 
       'Фантастический город. Детская площадка',
       'Зал игp', 'Вертолет Ми-8', 'Смотровая площадка в Ландшафтном парке',
       'Детская площадка с беседками на Липовой аллее',
       'Детская площадка в парке «Останкино»', 'Детская площадка между Липовой и Спортивной аллеями',
       'Детская площадка для занятий паркуром', 'Космос. Детская площадка', 'Выводной круг',
       'Самолет Як-42', 'Арт-композиция «Птицы»', 'Кинетическая скульптура «Армадилло»',
       'Детская площадка за павильоном №34', 'Башня-колонна Эля Лисицкого',
       'Водная дорога и пляж', 'Павильон №12', 'Сад молодоженов', 'Узбекский парк',
       'Северный сквер', 'Игровая площадка Capital Group',
       'Сочинская горка', '14 фонтанов Центральной аллеи',
       'Южный розарий', 'Сирингарий', 'Щелкановская аллея',
       'Минский сквер', 'Нальчикский участок', 'Кисловодский участок',
       'Колхозный сад', 'Ялтинский участок', 'Днепровский участок',
       'Рязанский участок', 'Транзитный сквер', 'Омский сквер',
       'Театральный зал', 'Зал для чтения',
       'Сквер у главного входа в парк «Останкино» со стороны телецентра',
       'Арка Южного входа', 'Арка Северного входа']

for_bike = [
    'Павильон №34. Центр «Космонавтика и авиация»',
       'Павильон №58. Музей славянской письменности «Слово»',
       'Павильон «Рабочий и колхозница»',
       'Павильон №5. Музей городского хозяйства Москвы',
       'Павильон №36. Музей кино',
       'Павильон №53-54. Музей Гаража особого назначения ФСО России',
       'Интерактивный музейный комплекс «Буран»','Павильон №17. Галерея «Лес»',
       'Арка Главного входа', 'Павильон №2. Робостанция',
       'Павильон №26. Музей Транспорта Москвы',
       'Павильон №13. Музей Востока ',
       'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
       'Павильон №30. Центр современных биотехнологий Музей «Биотех»',
       'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
       'Павильон №18. Выставочно-торговый центр Республики Беларусь',
       'Павильон №62. Международный центр балета',
       'Павильон №68. Выставочно-торговый центр Республики Армения',
       'Экспозиция экосистем ', 'Экотропа',
       '«Техноград». Инновационно-образовательный комплекс',
       'Павильон «Умный город»',
       'Павильон №10. Выставочно-торговый центр Республики Молдова',
       'Павильон №11. Выставочно-торговый центр Республики Казахстан ',
       'Павильон №25 «Нефть»',
       'Павильон №29. Эколого-просветительский центр «Цветоводство»',
       'Павильон №23. «Москвариум»',
       'Павильон №71. Дворец госуслуг (МФЦ) ',
       'Павильон №40. Школа пекарей',
       'Павильон №60. Потребительская кооперация',
       'Фонтан «Мальчик с рыбой»', 'Фонтан «Каменный цветок»',
       'Фонтан «Золотой колос»', 'Фонтан «Дружба народов»', 
       'Понтонный променад',
       'Зеленый бельведер в парке «Останкино»',
       'Большая прогулочная аллея', 'Большая картина полей',
       'Дворцовый (Останкинский) пруд',
       'Каскад Каменских прудов (2-й пруд)',
       'Каскад Каменских прудов (3-й пруд)',
       'Каскад Каменских прудов (4-й пруд)', 'Садовый пруд',
       'Северный розарий', 'Сквер юннатов', 'Дубовая роща',
       'Мичуринский сад', 'Медовый домик',
       'Латвийский сквер', 'Летняя эстрада в парке «Останкино»',
       'Мемориал-стела «Врата победы»', 'Макет ракеты-носителя «Восток»',
       'Зал игp', 'Вертолет Ми-8', 'Смотровая площадка в Ландшафтном парке',
       'Самолет Як-42', 'Арт-композиция «Птицы»', 'Кинетическая скульптура «Армадилло»',
       'Башня-колонна Эля Лисицкого',
       'Водная дорога и пляж', 'Павильон №12', 'Сад молодоженов', 'Узбекский парк',
       'Северный сквер', 'Сочинская горка', '14 фонтанов Центральной аллеи',
       'Южный розарий', 'Сирингарий', 'Щелкановская аллея',
       'Минский сквер', 'Нальчикский участок', 'Кисловодский участок',
       'Колхозный сад', 'Ялтинский участок', 'Днепровский участок',
       'Рязанский участок', 'Транзитный сквер', 'Омский сквер',
       'Театральный зал', 'Зал для чтения',
       'Сквер у главного входа в парк «Останкино» со стороны телецентра',
       'Арка Южного входа', 'Арка Северного входа'
]

In [6]:
def reversed_flag(x, arr):
    if x in arr:
        return 0
    else:
        return 1
    
def get_flag(x, arr):
    if x in arr:
        return 1
    else:
        return 0

df['children_flag'] = df['title'].apply(reversed_flag, args=[not_for_children])
df['date_flag'] = df['title'].apply(reversed_flag, args=[not_for_date])
df['family_flag'] = df['title'].apply(reversed_flag, args=[not_for_family])
df['animals_flag'] = df['title'].apply(get_flag, args=[for_animals])
df['bike_flag'] = df['title'].apply(get_flag, args=[for_bike])

### Интересы

In [7]:
promenade = ['Фонтан «Мальчик с рыбой»', 'Фонтан «Каменный цветок»', 'Фонтан «Золотой колос»', 'Экотропа',
             'Фонтан «Дружба народов»', 'Зеленый лабиринт', 'Понтонный променад', 'Зеленый бельведер в парке «Останкино»',
             'Большая прогулочная аллея', 'Большая картина полей', 'Дворцовый (Останкинский) пруд',
             'Каскад Каменских прудов (2-й пруд)', 'Каскад Каменских прудов (3-й пруд)', 'Каскад Каменских прудов (4-й пруд)',
             'Садовый пруд', 'Северный розарий', 'Сквер юннатов', 'Дубовая роща', 'Латвийский сквер',
             'Летняя эстрада в парке «Останкино»', 'Смотровая площадка в Ландшафтном парке', 'Арт-композиция «Птицы»',
             'Кинетическая скульптура «Армадилло»', 'Башня-колонна Эля Лисицкого', 'Водная дорога и пляж',
             'Сад молодоженов', 'Узбекский парк', 'Северный сквер', 'Сочинская горка',
             '14 фонтанов Центральной аллеи', 'Южный розарий', 'Сирингарий', 'Мичуринский сад',
             'Летний кинотеатр-лекторий', 'Щелкановская аллея', 'Минский сквер',
             'Нальчикский участок', 'Кисловодский участок', 'Колхозный сад',
             'Ялтинский участок', 'Днепровский участок', 'Рязанский участок',
             'Транзитный сквер', 'Омский сквер', 'Театральный зал', 'Экспозиция экосистем ',
             'Сквер у главного входа в парк «Останкино» со стороны телецентра', 'Ялтинский участок',
             'Арка Южного входа', 'Арка Северного входа', 'Выводной круг', 'Зал для чтения']

activity = ['Универсальная спортивная площадка для игры в мини-футбол и стритбол', 'Sky Town. Высотный веревочный парк ',
            'Павильон №22. Скалолазный центр BigWall ',
            'SUP-станция «ПОРТ»', 'Парк интерактивных развлечений Engage VR', 'Колесо обозрения «Солнце Москвы»',
            'Площадка для воркаута', 'Площадка для воркаута рядом со стадионом', 'Танцплощадка в парке «Останкино»']


nature = ['Павильон №17. Галерея «Лес»', 'Городская ферма', 'Павильон №28. Эколого-просветительский центр «Пчеловодство»',
          'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
          'Павильон №29. Эколого-просветительский центр «Цветоводство»', 'Павильон №23. «Москвариум»',
          'Павильон №43. Показательная конюшня Центра национальных конных традиций', 'Цветочная мастерская «Май-Май»',
          'Дом бабочек', 'Сирингарий', 'Южный розарий', 'Северный розарий']

science = ['Павильон №2. Робостанция', 'Павильон №28. Эколого-просветительский центр «Пчеловодство»',
           'Павильон №25 «Нефть»', 'Павильон №30. Центр современных биотехнологий Музей «Биотех»',
           '«Техноград». Инновационно-образовательный комплекс',
           'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
           'Павильон №29. Эколого-просветительский центр «Цветоводство»']

national = ['Павильон №58. Музей славянской письменности «Слово»', 'Павильон «Рабочий и колхозница»',
            'Фонтан «Дружба народов»', 'Павильон №13. Музей Востока ',
            'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
            'Павильон №18. Выставочно-торговый центр Республики Беларусь',
            'Павильон №68. Выставочно-торговый центр Республики Армения',
            'Павильон №10. Выставочно-торговый центр Республики Молдова',
            'Павильон №11. Выставочно-торговый центр Республики Казахстан ',
            'Павильон №43. Показательная конюшня Центра национальных конных традиций']

workshop = ['Мастерская «Симфония фарфора»', 'Медиа Школа Ольги Спиркиной Ostankino.tv',
            'Медовый домик', 'Мануфактура «Дымов керамика»', 'Архитектурно-художественная студия Artdomus',
            'Студия мозаики «Сад Гранат»', 'Школа кондитерского мастерства Rustik Bakery', 'Цветочная мастерская «Май-Май»',
            'Студия витражей и мозаики Moscow ART Glass', 'Косметическая лаборатория Pure Love',
            'Мастерская «Особая керамика»', 'Павильон №40. Школа пекарей',
            'Карамельная мануфактура', 'Мастерская елочных игрушек Christmas Art', 'Школа сыроделия Олеси Шевчук ',
            'Мастерская и школа стекла', 'Творческая мастерская Марии Багмет',
            'Архитектурно-производственная мастерская Uniquely',
            'Кожевенная мастерская Bottega Berto']

creation = ['Павильон №9. Театр сказок', 'Дом культуры', 'Павильон №36. Музей кино',
            'Павильон №7. «Союзмультпарк»', 'Павильон «Книги» РОСИЗО', 'Павильон №62. Международный центр балета',
            'Студия витражей и мозаики Moscow ART Glass', 'Творческая мастерская Марии Багмет',
            'Арт-резиденция музея «Гараж»', 'Центр креативного арт-дизайна La Atmosfera', 
           'Студия мозаики «Сад Гранат»', 'Архитектурно-художественная студия Artdomus']

kids = ['Павильон №7. «Союзмультпарк»', 'Павильон №2. Робостанция', 'Дом комиксов «TAKAPULTA»',
        'Детская площадка на главной аллее парка «Останкино»', 'Детская площадка за павильоном №34',
        'Игровая площадка, адаптированная для детей с ограниченными возможностями',
        'Детская площадка с беседками на Липовой аллее',
        'Детская площадка в парке «Останкино»', 'Фантастический город. Детская площадка',
        'Детская площадка между Липовой и Спортивной аллеями',
        'Детская площадка для занятий паркуром', 'Нефтеград. Детская интерактивная площадка',
        'Космос. Детская площадка', 'Павильон №55. Музей оптических иллюзий ',
        'Детская площадка за павильоном №34', 'Карамельная мануфактура',
        'Мастерская елочных игрушек Christmas Art', '«Детское посольство»', 'Студия детских настольных игр Dodoland',
        'Игровая площадка Capital Group', 'Лаборатория мороженого'
        ]

tech = ['Павильон №34. Центр «Космонавтика и авиация»', 'Павильон №53-54. Музей Гаража особого назначения ФСО России',
        'Интерактивный музейный комплекс «Буран»', 'Павильон №2. Робостанция', 'Вертолет Ми-8', 'Истребитель Су-27',
        'Павильон №26. Музей Транспорта Москвы', 'Макет ракеты-носителя «Восток»', 'Самолет Як-42',
        'Павильон «Умный город»', 'Павильон «Умный город»', 'Павильон №30. Центр современных биотехнологий Музей «Биотех»',
        '«Техноград». Инновационно-образовательный комплекс']

about_russia = ['Павильон №5. Музей городского хозяйства Москвы', 'Павильон №26. Музей Транспорта Москвы',
                'Павильон №57. «Россия — моя история»', 'Макет Москвы', 'Павильон №71. Дворец госуслуг (МФЦ) ',
                'Павильон №58. Музей славянской письменности «Слово»', 'Павильон «Умный город»',
                'Павильон «Рабочий и колхозница»', 'Музей ВДНХ', 'Павильон №12']

In [8]:
df['promenade'] = df['title'].apply(get_flag, args=[promenade])
df['activity'] = df['title'].apply(get_flag, args=[activity])
df['nature'] = df['title'].apply(get_flag, args=[nature])
df['science'] = df['title'].apply(get_flag, args=[science])
df['national'] = df['title'].apply(get_flag, args=[national])
df['workshop'] = df['title'].apply(get_flag, args=[workshop])
df['creation'] = df['title'].apply(get_flag, args=[creation])
df['kids'] = df['title'].apply(get_flag, args=[kids])
df['tech'] = df['title'].apply(get_flag, args=[tech])
df['about_russia'] = df['title'].apply(get_flag, args=[about_russia])

In [9]:
df.loc[df['title'].isin(['Кафе «Свадьба соек»', 'Павильон №39. Кафе «Фонтан VIEW»',
       'Кафе «Андерсон на даче»', 'Кафе «Пироги»', 'Фудкорт между павильонами №64 и №66',
       'Кафе «Русский чай»', 'Пекарня «Трдельник»', 'Лаборатория мороженого',
       'Кафе «Мороженое»', 'Кафе «Космос»',
       'Ресторан быстрого питания «Вкусно\xa0— и точка»',
       'Кафе «Спецбуфет особого назначения»', 'Кафе «Те самые пончики»', 'Кафе «Ребрышки и крылышки»',
       'Киоск Lukumades ', 'Кафе «Урожай»', 'Кофейня Vigor Maker',
       'Кафе Mamma Italia', 'Кафе «Галерея»',
       'Кофейная лаборатория «Смарт кофе»', 'Кафе «РОСИЗО»',
       'Cafe & Пельменная', 'Кафе «Ласточка»', 'Кафе «Арцеу»',
       'Кафе «Старое место»', 'Кафе «Вареничная №1»', 'Чайхана «Долина»',
       'Таверна «У околицы»', 'Кафе «Пельменя 61»',
       'Кафе «Те самые пончики»', 'Кафе «Дельфин»', 'Кофе-поинт «Акжера»',
       'Бульбяная', 'Кофейня «Шоколадница»']), 'type'] = 'Кафе'

df.loc[df['title'].isin(['Ресторан «Московское небо»', 'Ресторан «Арарат»',
       'Ресторан «Ферма»', 'Ресторан «Остерия Трио»',
       'Ресторан «Оттепель»', 'Ресторан «Рыбацкая деревня»',
       'Ресторан «Дастархан»','Ресторан «Домик в саду»', 'Ресторан Ricci Capricci ',
       'Ресторан «Все лень»', 'Ресторан Shevol ', 'Ресторан «Лав»']), 'type'] = 'Ресторан'

df.loc[df['title'].isin(['Корейский стрит-фуд «Пян-се»', 'Киоск «Грамотный кофе»',
       'Киоск «Грамотный кофе»', 'Домик «Колбаски гриль, чуррос»',
       'Doner One', 'Кафе «Shawarma house»',
       'Фудкорт в комплексе «Техноград»', 'Кофейня Park Point',
       'Restomarket']), 'type'] = 'Стрит-фуд'

df.loc[df['type'].isin(['Кафе', 'Ресторан', 'Стрит-фуд']), 'type_service'] = 'Еда'
df.loc[df['type'] == 'Туалеты', 'type_service'] = 'Туалеты'
df.loc[df['type'] == 'Прокат', 'type_service'] = 'Прокат'
df.loc[df['type'] == 'Остановка', 'type_service'] = 'Остановка'

In [10]:
df.loc[df['title'].isin([
    'Памятник В.И. Ленину', 'Арка Южного входа', 'Арка Северного входа',
    'Памятник изобретателю телевизора В.К. Зворыкину',
    'Памятник И.В. Мичурину', 'Мемориал-стела «Врата победы»',
    'Фонтан «Мальчик с рыбой»', 'Фонтан «Каменный цветок»',
    'Фонтан «Золотой колос»', 'Арт-композиция «Птицы»',
    'Кинетическая скульптура «Армадилло»', 'Башня-колонна Эля Лисицкого',
    'Памятный знак «Работникам ВСХВ, погибшим на фронтах Великой Отечественной войны 1941-1945 гг.»']), 'duration'] = 5

df.loc[df['title'].isin(
    ['Фонтан «Дружба народов»', 'Макет ракеты-носителя «Восток»']), 'duration'] = 10

df.loc[df['title'].isin([
    'Северный сквер', 'Игровая площадка Capital Group',
       'Сочинская горка', '14 фонтанов Центральной аллеи', 'Водная дорога и пляж',
       'Минский сквер', 'Павильон №12', 'Сад молодоженов', 'Театральный зал', 'Зал для чтения',
       'Летняя эстрада в парке «Останкино»', 'Сквер у главного входа в парк «Останкино» со стороны телецентра',
       'Южный розарий', 'Сирингарий', 'Щелкановская аллея', 'Дубовая роща',
       'Мичуринский сад', 'Сквер юннатов', 'Латвийский сквер', 'Летняя эстрада в парке «Останкино»'
       'Минский сквер', 'Нальчикский участок', 'Кисловодский участок',
       'Колхозный сад', 'Ялтинский участок', 'Днепровский участок',
       'Рязанский участок', 'Транзитный сквер', 'Омский сквер', 'Дворцовый (Останкинский) пруд',
       'Каскад Каменских прудов (2-й пруд)', 'Выводной круг', 'Узбекский парк',
       'Каскад Каменских прудов (3-й пруд)', 'Экспозиция экосистем ', 'Экотропа',
       'Каскад Каменских прудов (4-й пруд)', 'Садовый пруд']), 'duration'] = 20

df.loc[df['title'].isin([
       'Павильон №58. Музей славянской письменности «Слово»',
       'Павильон «Рабочий и колхозница»', 'Дом культуры',
       'Павильон №53-54. Музей Гаража особого назначения ФСО России',
       'Интерактивный музейный комплекс «Буран»',
       'Павильон №7. «Союзмультпарк»', 'Павильон «Книги» РОСИЗО',
       'Павильон №17. Галерея «Лес»',
       'Арка Главного входа', 'Павильон №2. Робостанция',
       'Павильон №26. Музей Транспорта Москвы',
       'Павильон №28. Эколого-просветительский центр «Пчеловодство»',
       'Макет Москвы', 'Шахматный клуб', 'Павильон №13. Музей Востока ',
       'Павильон №14. Выставочно-торговый центр Азербайджанской Республики',
       'Павильон №30. Центр современных биотехнологий Музей «Биотех»',
       'Павильон №31 «Геология». Биологический музей им. К.А.Тимирязева',
       'Павильон №18. Выставочно-торговый центр Республики Беларусь',
       'Павильон №62. Международный центр балета',
       'Павильон №68. Выставочно-торговый центр Республики Армения',
       'Центр креативного арт-дизайна La Atmosfera',
       '«Техноград». Инновационно-образовательный комплекс',
       'Павильон «Умный город»',
       'Павильон №10. Выставочно-торговый центр Республики Молдова',
       'Павильон №11. Выставочно-торговый центр Республики Казахстан ',
       'Павильон №22. Скалолазный центр BigWall ', 'Павильон №25 «Нефть»',
       'Павильон №29. Эколого-просветительский центр «Цветоводство»',
       'Павильон №71. Дворец госуслуг (МФЦ) ',
       'Павильон №55. Музей оптических иллюзий ',
       'Павильон №43. Показательная конюшня Центра национальных конных традиций',
       'Павильон №40. Школа пекарей',
       'Павильон №60. Потребительская кооперация',
       'Арт-резиденция музея «Гараж»', 'Читальня', 'Читальня', 'Читальня',
       'Читальня', 'Читальня', 'Читальня', 'Читальня', 'Читальня',
       'Читальня', 'Читальня',
       'Нефтеград. Детская интерактивная площадка', 'Зеленый лабиринт',
       'Лаборатория мороженого', 'Понтонный променад',
       'Зеленый бельведер в парке «Останкино»',
       'Большая прогулочная аллея', 'Большая картина полей',
       'Площадка для воркаута',
       'Площадка для воркаута рядом со стадионом',
       'Северный розарий',
       'Универсальная спортивная площадка для игры в мини-футбол и стритбол',
       'Танцплощадка в парке «Останкино»', 'Мастерская «Симфония фарфора»',
       'SUP-станция «ПОРТ»', 'Медиа Школа Ольги Спиркиной Ostankino.tv',
       'Парк интерактивных развлечений Engage VR', 'Медовый домик',
       'Мануфактура «Дымов керамика»',
       'Архитектурно-художественная студия Artdomus',
       'Детская площадка на главной аллее парка «Останкино»',
       'Дом комиксов «TAKAPULTA»',
       'Игровая площадка, адаптированная для детей с ограниченными возможностями',
       'Фантастический город. Детская площадка',
       'Студия мозаики «Сад Гранат»', 'Зал игp', 'Вертолет Ми-8',
       'Школа кондитерского мастерства Rustik Bakery',
       'Цветочная мастерская «Май-Май»',
       'Студия витражей и мозаики Moscow ART Glass',
       'Косметическая лаборатория Pure Love',
       'Мастерская «Особая керамика»',
       'Смотровая площадка в Ландшафтном парке',
       'Детская площадка с беседками на Липовой аллее',
       'Детская площадка в парке «Останкино»',
       'Детская площадка между Липовой и Спортивной аллеями',
       'Детская площадка для занятий паркуром',
       'Космос. Детская площадка', 'Истребитель Су-27',
       'Самолет Як-42', 'Детская площадка за павильоном №34',
       'Колесо обозрения «Солнце Москвы»', 'Карамельная мануфактура',
       'Мастерская елочных игрушек Christmas Art', '«Детское посольство»',
       'Школа сыроделия Олеси Шевчук ', 'Мастерская и школа стекла',
       'Творческая мастерская Марии Багмет',
       'Студия детских настольных игр Dodoland',
       'Архитектурно-производственная мастерская Uniquely',
       'Кожевенная мастерская Bottega Berto', 'Дом бабочек']), 'duration'] = 30

df.loc[df['title'].isin([
    'Павильон №34. Центр «Космонавтика и авиация»', 'Музей ВДНХ', 'Sky Town. Высотный веревочный парк ',
    'Павильон №36. Музей кино', 'Городская ферма', 'Павильон №5. Музей городского хозяйства Москвы']), 'duration'] = 40

df.loc[df['title'].isin(
    ['Павильон №57. «Россия — моя история»']), 'duration'] = 70

df.loc[df['title'].isin(['Павильон №23. «Москвариум»']), 'duration'] = 90

### Векторизация объектов

In [11]:
%%time
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from pymystem3 import Mystem

m = Mystem()

def full_preprocess(text):

    text = BeautifulSoup(text, 'lxml').text
    stop_words = set(stopwords.words('russian'))

    def delete_stop_words(text):

        filtered_tokens = []
        for token in text.split():
            if token not in stop_words:
                filtered_tokens.append(token)

        return ' '.join(filtered_tokens)

    def final_deleting(text):
        alphabet = 'abcdefghijklmnopqrstuvwxyzабвгдежзийклмнопрстуфхцчшщъыьэюя '

        text = text.replace("ё", "е")
        words = ''.join([[" ", i][i in alphabet] for i in text]).split()
        return ' '.join(words)

    text = delete_stop_words(text.lower())

    return final_deleting(text)


df['processed_preview'] = df['preview_text'].astype(str).apply(full_preprocess)
df['processed_title'] = df['title'].astype(str).apply(full_preprocess)

def process(value):
    if 'скидки' in value:
        return value[value.find('скидки') + 7:]
    else:
        return value
    
df['processed_preview'] = df['processed_preview'].apply(process)

CPU times: user 1.22 s, sys: 945 ms, total: 2.16 s
Wall time: 1.72 s


In [12]:
import compress_fasttext

model = compress_fasttext.models.CompressedFastTextKeyedVectors.load('model.bin')

df['vector_text'] = df['processed_preview'].apply(model.get_vector)
df['vector_title'] = df['processed_title'].apply(model.get_vector)

In [13]:
df.shape

(378, 60)

In [14]:
js = df.transpose().to_json(orient='columns')
db = json.loads(js)

with open("db.json", "w") as outfile:
    outfile.write(js)